In [ ]:
%load_ext autoreload
%autoreload 2

# Exercise 1

<img src="./images/01.png" width=800>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
import torch.optim as optim
import os
import mlflow
from torchinfo import summary
import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader, random_split
from utils import train_network, set_seed

/home/spakdel/my_projects/Books/Inside-Deep-Learning/Exercises_InsideDeepLearning/Chapter_08/utils.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [ ]:
os.environ['MLFLOW_TRACKING_URI'] = './mlruns08_1'
mlflow.set_tracking_uri(os.environ.get('MLFLOW_TRACKING_URI'))

In [ ]:
mlflow.set_experiment('Exercise08_1')

<Experiment: artifact_location='/home/spakdel/my_projects/Books/Inside-Deep-Learning/Exercises_InsideDeepLearning/Chapter_07/mlruns07_1/143507330168611334', creation_time=1750415411076, experiment_id='143507330168611334', last_update_time=1750415411076, lifecycle_stage='active', name='Exercise07_1', tags={}>

In [ ]:
torch.backends.cudnn.deterministic = True
set_seed(42)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

## Dataset and DataLoader

In [ ]:
class AutoencodDataset(Dataset):
    def __init__(self, dataset):
        super().__init__()
        self.dataset = dataset
    def __len__(self):
        return len(self.dataset)
    def __getitem__(self, index):
        # x, y = self.dataset.__getitem__(index)
        x, _ = self.dataset[index]
        return  x, x

In [ ]:
train_data = AutoencodDataset(torchvision.datasets.MNIST("./data", train=True, transform=transforms.ToTensor(), download=True))
test_data_xy = torchvision.datasets.MNIST("./data", train=False, transform=transforms.ToTensor(), download=True)
test_data_xx = AutoencodDataset(test_data_xy)
batch_size = 128
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data_xx, batch_size=batch_size)

## Model

In [ ]:
W = 28
H = 28
D = W * H
C =1
classes = 10

### Convolutional Autoencoder: without pooling and convtranspose

In [ ]:
def conv_block(in_channels, out_channels, kernel_size=3, stride=1, padding=None):
    if padding is None:
        # Calculate padding to maintain spatial dimensions for odd kernel sizes with stride 1
        padding = (kernel_size - 1) // 2 
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding),
        nn.BatchNorm2d(out_channels),
        nn.ReLU()
    )

In [ ]:
in_filters = 64
encoder_cnn = nn.Sequential(    
    conv_block(C, in_filters, kernel_size=3), 
    conv_block(in_filters, in_filters*2, kernel_size=3),
    nn.Conv2d(in_filters*2, in_filters*4, kernel_size=3, stride=1, padding=1),

)

decoder_cnn = nn.Sequential(
    conv_block(in_filters*4, in_filters*2, kernel_size=3),
    conv_block(in_filters*2, in_filters, kernel_size=3),
    nn.Conv2d(in_filters, C, kernel_size=3, stride=1, padding=1),
    nn.Sigmoid(),
)

# Combined Autoencoder
without_pooltrans_encod_decode_cnn = nn.Sequential(
    encoder_cnn,
    decoder_cnn,
)

### Convolutional Autoencoder: with pooling and convtranspose

In [ ]:
def conv_block(in_channels, out_channels, kernel_size=3, stride=1, padding=None):
    if padding is None:
        # Calculate padding to maintain spatial dimensions for odd kernel sizes with stride 1
        padding = (kernel_size - 1) // 2 
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding),
        nn.BatchNorm2d(out_channels),
        nn.ReLU()
    )

In [ ]:
in_filters = 64
encoder_cnn = nn.Sequential(
    conv_block(C, in_filters, kernel_size=3), 
    nn.MaxPool2d((2, 2)),
    conv_block(in_filters, in_filters*2, kernel_size=3),
    nn.MaxPool2d((2, 2)),
    nn.Conv2d(in_filters*2, in_filters*4, kernel_size=3, stride=1, padding=1),

)

decoder_cnn = nn.Sequential(
    nn.ConvTranspose2d(in_filters*4, in_filters*2, (3,3), padding=1, output_padding=1, stride=2),
    nn.ReLU(),
    nn.ConvTranspose2d(in_filters*2, in_filters, (3,3), padding=1, output_padding=1, stride=2),
    nn.ReLU(),
    nn.Conv2d(in_filters, C, kernel_size=3, stride=1, padding=1),
    nn.Sigmoid(),
)

# Combined Autoencoder
with_pooltrans_encod_decode_cnn = nn.Sequential(
    encoder_cnn,
    decoder_cnn,
)

## Training

In [ ]:
loss_func = nn.MSELoss()
epochs = 30
params = {
    'device': device,
    'loss_func': loss_func.__class__.__name__,
    'epochs': epochs,
    'batch_size': batch_size,
    }

In [ ]:
models = {
    'without_pooltranspose': without_pooltrans_encod_decode_cnn,
    'with_pooltranspose': with_pooltrans_encod_decode_cnn
}

In [ ]:
for experiment, model in models.items():
    params['experiment'] = experiment
    optimizer = optim.AdamW(model.parameters())
    params['optimizer'] = optimizer.defaults
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params_all = sum(p.numel() for p in model.parameters())
    params['trainable_params'] = trainable_params
    params['total_params_all'] = total_params_all
    with open('model_summary.txt', 'w') as f:
        f.write(str(summary(model, inpt_size=(batch_size, C, 28, 28))))
    with mlflow.start_run(nested=True, run_name=experiment):
        mlflow.log_artifact('model_summary.txt')
        mlflow.log_params(params)

        results = train_network(
            model=model,
            optimizer=optimizer,
            loss_func=loss_func,
            train_loader=train_loader,
            valid_loader=test_loader,
            epochs=epochs,
            device=device,
            # checkpoint_file_save='model.pth',
            
        )

## Results

In [ ]:
def evaluate_and_plot(models_dict):
    """Evaluate models on test data and visualize the reconstructions."""
    # Fetch a single batch of test images
    test_images, _ = next(iter(test_loader))
    test_images = test_images.to(device)
    
    reconstructions = {}
    for name, model in models_dict.items():
        model.eval()
        with torch.no_grad():
            reconstructions[name] = model(test_images).cpu()
    
    # Plotting
    n_images_to_show = 8
    num_models = len(models_dict)
    fig, axes = plt.subplots(num_models + 1, n_images_to_show, figsize=(n_images_to_show * 1.5, (num_models + 1) * 1.5))
    
    # Plot original images
    for i in range(n_images_to_show):
        ax = axes[0, i]
        ax.imshow(test_images[i].cpu().squeeze(), cmap='gray')
        ax.set_xticks([])
        ax.set_yticks([])
        if i == 0:
            ax.set_ylabel("Original", fontsize=12)
            
    # Plot reconstructions for each model
    row_idx = 1
    for name, recon_imgs in reconstructions.items():
        for i in range(n_images_to_show):
            ax = axes[row_idx, i]
            ax.imshow(recon_imgs[i].squeeze(), cmap='gray')
            ax.set_xticks([])
            ax.set_yticks([])
            if i == 0:
                ax.set_ylabel(name, fontsize=12)
        row_idx += 1
        
    plt.tight_layout()
    plt.suptitle("Autoencoder Reconstructions Comparison", fontsize=16, y=1.02)
    plt.show()


In [ ]:
evaluate_and_plot(trained_models)